<a href="https://colab.research.google.com/github/diloper/SRC_Cloud/blob/colab/random_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".

# !ls "/content/drive/MyDrive/RD_程式開發/SRC_deploy"
# import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks') 

# !pip install p2j

# !p2j random_test.py -o

In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
import re , os , sqlite3
import pandas as pd
# import read_file_analyze as ka
from datetime import date, timedelta, datetime

In [ ]:
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

In [ ]:
def find_SRC_negative_v2(dir,stockid,boundary=300):
#     now = datetime.now()
#     Today = now.strftime("%Y-%m-%d")
#     print(Today)
#     current_date = (now-timedelta(days=120)).isoformat()
    price_db_name=dir+str(stockid)+".sqlite"
    if os.path.isfile(price_db_name) is not True:
        return
    price_db_handler = sqlite3.connect(price_db_name)
    df = pd.read_sql_query('SELECT * from (SELECT OHLC.Date,OHLC.Close, SRC.M20 from SRC INNER JOIN OHLC ON SRC.Date = OHLC.Date order by SRC.Date desc LIMIT "'+str(boundary)+'") order by date ASC',price_db_handler)
    df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    price_db_handler.close()
    df['SRC M20 5dt']=(df['M20']-df['M20'].shift(5))/5
#   抓取SRC 5dt連續性
    df['SRC 5dt_G_R']=df['SRC M20 5dt']/abs(df['SRC M20 5dt'])
    df['5dt_G_R_sum']=df['SRC 5dt_G_R'].rolling(window=10,min_periods=10).sum()
#     df.loc[df['M20'] <0&, 'name_match'] = 'Match'
    date_before = date.today() - timedelta(days=boundary)
#     print(date_before)
#    df.loc[(df['M20']<0) & (df['M20']>=-0.1)& (df['5dt_G_R_sum']>8) & (df['Date'] > str(date_before)),'RRR']=1
# 可調整 SRC 條件
    A=df.loc[(df['M20']<0) & (df['M20']>=-0.1)& (df['5dt_G_R_sum']>8) & (df['Date'] > str(date_before)) ]
#    A=df.loc[df['RRR']==1]
    
    
#     if A.shape[0]>0:
#         print(A.head())
        
#         print(A.head())
#         A.to_csv(dir+str(stockid)+'SRC_dt.csv', index=False)
    return A

In [ ]:
def find_SRC_negative(dir,stockid,condition=True):
#     now = datetime.now()
#     Today = now.strftime("%Y-%m-%d")
#     print(Today)
#     current_date = (now-timedelta(days=120)).isoformat()
    price_db_name=dir+str(stockid)+".sqlite"
    if os.path.isfile(price_db_name) is not True:
        return
    price_db_handler = sqlite3.connect(price_db_name)
    df = pd.read_sql_query('SELECT OHLC.Date,OHLC.Close, SRC.M20 from SRC INNER JOIN OHLC ON SRC.Date = OHLC.Date order by SRC.Date ASC',price_db_handler)
    df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    price_db_handler.close()
    count_list=[]
    date_st_list=[]
    prc_st_list=[]
    prc_st_20MA_list=[]
    date_ed_list=[]
    prc_ed_list=[]
    prc_ed_20MA_list=[]
    diff_20MA_list=[]
    stockid_list=[]
    one_M_price_list=[]
#         
    df['G_R_Cond']=df['M20']/abs(df['M20'])#定義正為1 負為-1
    df['G_R_Cond'] = df['G_R_Cond'].fillna(0)
    df['SRC M20 5dt']=(df['M20']-df['M20'].shift(5))/5
#   抓取SRC 5dt連續性
    df['SRC 5dt_G_R']=df['SRC M20 5dt']/abs(df['SRC M20 5dt'])
    df['5dt_G_R_sum']=df['SRC 5dt_G_R'].rolling(window=10,min_periods=10).sum()
#     df.loc[df['M20'] <0&, 'name_match'] = 'Match'
    date_before = date.today() - timedelta(days=20)
#     print(date_before)
    df.loc[(df['M20']<0) & (df['5dt_G_R_sum']>8) & (df['Date'] > str(date_before))& (df['M20']>=-0.1),'RRR']=1
    A=df.loc[df['RRR']==1]
    if A.shape[0]>0:
#         print(A.head())
        A.to_csv(dir+str(stockid)+'SRC_dt.csv', index=False)
    count_row = df.shape[0]
    compare1=2
    compare2=-1
    if condition==False:
        compare1=-2
        compare2=1
    a=1
    while(a<count_row):
#     for index in range(0,count_row-1):
#         a=index+1
        index=a-1
        B=int(df.loc[index,['G_R_Cond']].values[0])
        C=int(df.loc[a,['G_R_Cond']].values[0])
#         print('loop pre',a)
#        B-C=> 1-(-1)=2表示SRC開始由正轉負，反之 由負轉正
        
        re=B-C
        if  re==compare1 :
    #         print(index,a)
            
            count=1
            while(1):
                a=a+1
                if a < count_row and int(df.loc[a,['G_R_Cond']].values[0])  == compare2:
                    count=count+1
                else:
                    if a >= count_row:
                        a=count_row-1
#                     print(a)
                    break
            if count>20: # SRC<0 連續20次
                
                date_st=df.loc[index,['Date']].values[0]
                prc_st_20MA=df.loc[index,['20 Day MA']].values[0]
                prc_st=df.loc[index,['Close']].values[0]
#                 date_ed=date_ed=df.loc[a,['Date']].values[0]
                prc_ed_20MA=df.loc[a,['20 Day MA']].values[0]
                prc_ed=df.loc[a,['Close']].values[0]
                diff_20MA=(prc_ed_20MA-prc_st_20MA)/prc_ed_20MA
                date_ed=df.loc[a,['Date']].values[0]
                
                
                Next_M=a+30
                if a+30>= count_row:# 取得{D+30  or 最新的}price
                    Next_M=count_row-1
#                 one_M_price=df.loc[Next_M,['20 Day MA']].values[0]
                if diff_20MA < 0: # 20MA均線呈現下降
                    count_list.append(count)
                    date_st_list.append(date_st)
                    prc_st_list.append(prc_st)
                    prc_st_20MA_list.append("{:.2f}".format(prc_st_20MA))
                    
                    date_ed_list.append(date_ed)
                    prc_ed_list.append(prc_ed)
                    prc_ed_20MA_list.append("{:.2f}".format(prc_ed_20MA))
                    
                    diff_20MA_list.append("{:.4f}".format(diff_20MA))
                    stockid_list.append(str(stockid))
                    
                    one_M_price=df.loc[Next_M,['20 Day MA']].values[0]
                    one_M_price_list.append(one_M_price)
                    
                    
#                     A=df.loc[index:a,['Close diff 20 Day MA']]
#                     print(A,len(A))
#                     print(count,","
#                       ,date_st,",","{:.2f}".format(prc_st_20MA)
#                       ,",",date_ed,",","{:.2f}".format(prc_ed_20MA)
#                       ,",","{:.4f}".format(diff_20MA)
#                       ,",","{:.2f}".format(one_M_price),",",str(stockid))
                        
#                   Using lists in dictionary to create dataframe
        a=a+1
#     df['Close diff 20 Day MA']=(df['20 Day MA']-df['Close'])/df['Close']
#     df.to_csv('Close diff 20 Day MA.csv', index=False)
    dicty = {'count_list':count_list
             ,'date_st_list': date_st_list,'prc_st_list': prc_st_list,'prc_st_20MA_list': prc_st_20MA_list
             ,'date_ed_list': date_ed_list,'prc_ed_list': prc_ed_list,'prc_ed_20MA_list': prc_ed_20MA_list
             ,'diff_20MA_list': diff_20MA_list,'one_M_price_list':one_M_price_list,'stockid_list':stockid_list,} 
    rf = pd.DataFrame(dicty) 
    return rf

In [ ]:
def find_SRC_back_positive(df,str_index,boundary=90,continue_times=20,find_positive=True):
    # str_index=307
    end_index=str_index-boundary+2
    if end_index <= 0:
#         print(str_index)
#         print(end_index)
#         print(df.shape[0])
        return None,None
    compare1=-2
    compare2=1
    if find_positive is not True:
        compare1=2
        compare2=-1
#第一層while是要找出斜率  開始由正轉負，反之 由負轉正
    while(str_index > end_index):
        str_index=str_index-1
        index=str_index-1
        B=int(df.loc[str_index,['G_R_Cond']].values[0])
        C=int(df.loc[index,['G_R_Cond']].values[0])
    # print('loop pre',str_index)
    # B-C=> 1-(-1)=2表示SRC開始由正轉負，反之 由負轉正 修改"find_positive"
        diff=B-C
        if  diff==compare1 :
    #         print(index,str_index)
    #         print(df.loc[str_index,['Date']].values[0])
            count=1
#第二層while檢測SRC斜率的連續性，如果要容許不連續要再做修改
            while(str_index >= end_index):
                str_index=str_index-1
                if str_index > end_index and int(df.loc[str_index,['G_R_Cond']].values[0])  == compare2:
                    count=count+1
                else:
#                     if str_index >= end_index:
    #                     str_index=count_row-1
#                         print(str_index,count)
    #                     print(count)
                    break
            if count > continue_times:
#                 print(index)
#                 print(df.loc[index,['Date']].values[0])
                Date=df.loc[index,['Date']].values[0]
#                 print(Date)
                return Date,index
    return None,None
#                 print(df.loc[str_index+count,['Date']].values[0])
    #         else:
    #             print(count)

In[68]:

In [ ]:
def query_(dir,stockid,boundary,date):
    price_db_name=dir+str(stockid)+".sqlite"
    # if os.path.isfile(price_db_name) is not True:
    #     return
    price_db_handler = sqlite3.connect(price_db_name)
    tat='SELECT * FROM(SELECT OHLC.Date,OHLC.Close,SRC.M20 from  SRC INNER JOIN OHLC ON SRC.Date = OHLC.Date order by SRC.Date DESC )WHERE Date <="'+ str(date) +'"order by Date desc  LIMIT "'+str(boundary)+'"'
    # df = pd.read_sql_query('SELECT OHLC.Date,OHLC.Close, SRC.M20 from SRC INNER JOIN OHLC ON SRC.Date = OHLC.Date order by SRC.Date ASC',price_db_handler)
    # df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    df = pd.read_sql_query(tat,price_db_handler)
    price_db_handler.close()
    
    df=df.iloc[::-1]
    df.reset_index(drop=True,inplace=True)
    df['G_R_Cond']=df['M20']/abs(df['M20'])#定義正為1 負為-1
    df['G_R_Cond'] = df['G_R_Cond'].fillna(0)
    # priec M20 slope 
    df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    df['20 Day MA']=df['20 Day MA'].shift(1)
    df.loc[:,'20 Day MA_5dt']=((df['20 Day MA']-df['20 Day MA'].shift(5))/df['20 Day MA'].shift(5)).round(4)
    return df
# 分析SRC 於負的區間的時間與 股價變化
# Step 1透過find_SRC_negative 先找出 SRC 斜率 上升 且即將由負轉正
# Step 2 針對各別時間點，再往前搜尋最後一次SRC由正轉負的

In [ ]:
def show_period_MA20_status(dfG,df):
    for row in dfG.itertuples():
        str_index=getattr(row, 'str_index')
        end_index=getattr(row, 'end_index')
    #         print(getattr(row, 'str_date'))
        A=df.iloc[str_index:end_index]

    #  找出斜率|x|<=0.002 的發生點
    #         PR=A[A['20 Day MA_5dt']<=0.002 & A['20 Day MA_5dt']>=-0.002]
        cond1=A['20 Day MA_5dt']<=0.002
        cond2=A['20 Day MA_5dt']>=-0.002
    #         PR=A[A['20 Day MA_5dt']<=0.002]
        PR=A.loc[cond2 & cond1 ]
        head=A.iloc[0:1]
        tail=A.tail(1)
    #     data = []
    #     data.insert(0, head)
    # 檢查是否已存在，插入頭尾兩筆資料
        PR=pd.concat([head, PR], ignore_index=False)
        PR=pd.concat([PR,tail], ignore_index=False)
        # dropping duplicate values 
        PR.drop_duplicates(keep='first',inplace=True) 
        PR=continue_remove(PR)
        print(PR)

        # 依照區間進行分析曲線 升 降
        arr=PR[:].index.values
        # arr
        for i in range(len(arr)-1):
            print(arr[i],arr[i+1])
            str_index=arr[i]
            end_index=arr[i+1]
            tmp=df.iloc[str_index:end_index+1]
#             print(tmp)
        #     統計 20 Day MA_5dt 正負的次數?
            cond1=tmp['20 Day MA_5dt']>0.0
            positive=tmp.loc[ cond1 ]

            print('正:',positive.shape[0])<br>
            print('負:',tmp.shape[0]-positive.shape[0])    <br>
        print('----------------------')<br>
print(dfG)

In [ ]:
def modification_date(filename):
    t = os.path.getmtime(filename)
    R=datetime.fromtimestamp(t)
    return R.strftime("%Y-%m-%d")    
def local_file(last_folder,Nodir=True):
    # 歷遍資料夾下的的檔案名稱，符合後紀錄於local_files
    local_files=list()
    for root, dirs, files in os.walk(last_folder, topdown=True):
        for name in files:
            modify_time=modification_date(last_folder+'/'+name)
            name=name.replace("'","")
            if 'sqlite' in name and len(name) == 11:
                name=name.replace(".sqlite","")
                if Nodir is False:
                    data= (last_folder+'/'+name,modify_time)
                else:
                    data= (name,modify_time)
                local_files.append(data)
#                 file_time.append(modify_time)
    return local_files

In [ ]:
def bollinger_bands(data,bis=2):
#     data=copy.deepcopy(df)
    data['std']=data['Close'].rolling(window=20,min_periods=20).std()
    # data['std']=data['std'].shift(1)
#     data['5 Day MA Vol']=data['Volume'].rolling(window=5,min_periods=5).mean()
#     data['5 Day MA Vol']=data['5 Day MA Vol'].shift(1)
#     temp=[]
#     temp=data['Volume']/data['5 Day MA Vol']
#     data['Vol > 5 Day MA Vol']=temp.where(temp>=3)
#     data['5 Day MA Vol'].loc[0] = data['5 Day MA Vol'1].loc[1]
#     data['5 Day MA Vol'].drop([len(data['5 Day MA Vol'])])
    data['20 Day MA']=data['Close'].rolling(window=20,min_periods=20).mean()
    # data['20 Day MA']=data['20 Day MA'].shift(1)
    data['Upper Band']=data['20 Day MA']+bis*data['std']
    data['Lower Band']=data['20 Day MA']-bis*data['std']
    #布林帶寬
    data['bolling Bandwith']=(data['Upper Band']-data['Lower Band'])/data['20 Day MA']
#     data['bolling Bandwith 10MA']=data['bolling Bandwith'].rolling(window=10,min_periods=10).mean()
#     data['bolling Bandwith 5MA']=data['bolling Bandwith'].rolling(window=5,min_periods=5).mean()
    #10 布林帶寬變化
#     data['10 bolling Band std']=data['bolling Bandwith'].rolling(window=10,min_periods=10).std()
    #保留小數點後第二位
    data=data.round({'bolling Bandwith':2,'10 bolling Band std':2})
    
    #data["bolling Bandwith < 5%"]=data['bolling Bandwith'].where(data['bolling Bandwith']<0.05)
    return data

In[ ]:

In [ ]:
def forecast_Evaluate(dir,stockid,end_date,forecast_day=60):
    # search stock for future x day find max/min price 
    price_db_name=dir+str(stockid)+".sqlite"
    # if os.path.isfile(price_db_name) is not True:
    #     return
    price_db_handler = sqlite3.connect(price_db_name)
    df = pd.read_sql_query('SELECT Date,Close from OHLC where Date >="'+str(end_date)+'" order by Date ASC LIMIT "'+str(forecast_day)+'"',price_db_handler)
    # df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    if df.shape[0]<60:
        forecast_day=df.shape[0]
    # end_date
    # retrive max/min price  close data  forecast_day
    # df
    column = df['Close']
    max_index = column.idxmax()
    min_index=column.idxmin()
    # compare end date close price with max min
    Hdiff=(column[max_index]-column[0])/column[0]
    Ldiff=(column[min_index]-column[0])/column[0]
    result=False
    G=0.11
    H=0.2
    # 跌幅<G 漲幅>H
    diff=Ldiff
    if abs(Ldiff)<G and Hdiff>H:
#     if Hdiff>H:
        result=df.at[max_index,'Date']
        diff=Hdiff
    else:
        result=df.at[min_index,'Date']
    return result,diff,column[0]

In [ ]:
def break_bolling_low(dir,socketid,str_date,end_date):
#     forecast_day=120
#     socketid=2301
#     str_date='2020-01-10'
#     end_date='2020-04-01'
    # search stock for future x day find max/min price 
    price_db_name=dir+str(socketid)+".sqlite"
    # if os.path.isfile(price_db_name) is not True:
    #     return
    price_db_handler = sqlite3.connect(price_db_name)
    dfb = pd.read_sql_query('SELECT Date,Close from OHLC where Date <"'+str(str_date)+'" order by Date DESC LIMIT 19',price_db_handler)
    dfb.sort_values(by=['Date'], inplace=True)
    df1 = pd.read_sql_query('SELECT Date,Close from OHLC where  Date<="'+ str(end_date) +'"AND Date >="'+str(str_date)+'" order by Date ASC',price_db_handler)
    price_db_handler.close()
    
    df=pd.concat([dfb, df1])
    
    # df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
    bollinger_bands(df,bis=2)
#     print(df)
#     df.to_csv('out.csv', index=False) 
    # df.drop(['B', 'C'], axis=1)
    # df['']=
    F=df['bolling Bandwith'].tail(1).values[0]
    df.loc[df['Lower Band'] > df['Close'] , 'SING'] = 1 
    df=df.iloc[19:]
    df.reset_index(drop=True,inplace=True)
    # price_db_handler.close()
    if df.shape[0]<60:
        forecast_day=df.shape[0]
    # end_date
    # retrive max/min price  close data  forecast_day
    # df
#     column = df['Close']
#     max_index = column.idxmax()
#     min_index=column.idxmin()
#     # compare end date close price with max min
#     Hdiff=(column[max_index]-column[0])/column[0]
#     Ldiff=(column[min_index]-column[0])/column[0]
#     result=False
#     G=0.11
#     H=0.2
#     # 跌幅<G 漲幅>H
#     if abs(Ldiff)<G and Hdiff>H:
#         result=df.at[max_index,'Date']
#     print(socketid)
    #SING =1 Close price < Bolling Lower Band 
    condA=df['SING']==1
    # df
    E=df.loc[condA]
#     跌破布林通道底部占比
    P=0
    M20_direction=0
    if E.shape[0]>0:
        P=E.shape[0]/df.shape[0]
        A=E.at[E.head(1).index[0],'20 Day MA']
        B=E.at[E.tail(1).index[0],'20 Day MA']
        M20_direction = 1 if A>B else -1
    return P,E.shape[0],M20_direction,F

In [ ]:
def SRC_notify(oldfilename,df):
    new_df=df.loc[: ,['socketid','end_date']]

    # 有資料才比較
    new_df_A=new_df
    if new_df.shape[0] > 0 :
        # check if data exist
        if os.path.isfile(oldfilename) :
            old_df=pd.read_csv(oldfilename)
            counter=old_df.shape[0] 
            for row in old_df.itertuples():
                end_date=getattr(row, 'end_date')
                socketid=getattr(row, 'socketid')
                
                condition = new_df_A['socketid'] == str(socketid)
                Q=new_df_A.loc[condition]
                index_A=Q.loc[Q['end_date']==end_date]
                if index_A.shape[0]==1:
                    new_df_A=new_df_A.drop(index_A.index.tolist())
                    counter=counter-1 
#            當counter=0 則表示old_df行數全比較完畢        
            if len(new_df_A) == 0 and counter== 0:
                print('the same')
                return False
        print('save file')
        df.to_csv(oldfilename, encoding='utf-8',index=False)
        return True

In [ ]:
dir='./small_test/'
def main():
    oldfilename='SRCresult.csv'
    A=find_SRC_by_condition(dir=dir)
    B=SRC_notify(oldfilename=oldfilename,df=A)
#    C=SRC_notify(oldfilename=A,df=oldfilename)
    print(A)
# In[94]:

In [ ]:
if __name__ == '__main__':
    main()

Empty DataFrame
Columns: [socketid, str_date, str_index, end_date, end_index, during, forecast_Evaluate, end_close, price_diff, bolling Bandwith, Percentage_of_touch_bolling_bottom, times, M20_direction]
Index: []
2022-09-12
Empty DataFrame
Columns: [socketid, end_date, end_close]
Index: []


In [ ]:
import os
import platform

os.chdir('/content/drive/MyDrive/RD_程式開發/SRC_deploy') 
os.listdir() #確認目錄內容
A=local_file('./small_test')
A
platform.platform()
def test():
  stockid=1101
  price_db_name=dir+str(stockid)+".sqlite"
  if os.path.isfile(price_db_name) is not True:
      return
  price_db_handler = sqlite3.connect(price_db_name)
  df = pd.read_sql_query('SELECT * from financing ',price_db_handler)
  # df['20 Day MA']=df['Close'].rolling(window=20,min_periods=20).mean()
  price_db_handler.close()
  return df
test()

,buy,sell,yesterday_balance,today_balance,fin_cost,fin_maintenance_rate,Date
0,231.0,21.0,5563,5763,47.8247,159.9592,2022-04-29
1,151.0,98.0,5763,5791,47.7615,158.2516,2022-05-03
2,70.0,29.0,5791,5832,47.7345,158.8648,2022-05-04
3,21.0,46.0,5832,5807,47.7268,159.2397,2022-05-05
4,49.0,59.0,5807,5797,47.7048,157.5663,2022-05-06
5,218.0,176.0,5797,5839,47.5796,155.0034,2022-05-09
6,117.0,67.0,5839,5886,47.5063,153.8392,2022-05-10
7,252.0,127.0,5886,6011,47.3069,150.2600,2022-05-11
8,451.0,332.0,6011,6125,46.8528,145.1354,2022-05-12
9,191.0,158.0,6125,6156,46.6940,148.4845,2022-05-13
